## Run differr

Do pairwise comparisons between the strains.

In [1]:
# Get pairwise comparisons
import pandas as pd
run_to_ass = pd.read_csv(f'../output/run_to_assembly.csv')
run_to_ass

,Run,SampleName,Species,Strain,Name,AssemblyID
0,SRR13212615,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
1,SRR13212616,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
2,SRR13212618,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
3,SRR13212623,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
4,SRR13212624,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
...,...,...,...,...,...,...
79,SRR16608236,Bile1,Vibrio,RIMD,Vibrioparahaemolyticus-RIMD_2210633,GCF_000196095.1
80,SRR16608237,LB2,Vibrio,RIMD,Vibrioparahaemolyticus-RIMD_2210633,GCF_000196095.1
81,SRR16608238,LB1,Vibrio,RIMD,Vibrioparahaemolyticus-RIMD_2210633,GCF_000196095.1
82,SRR18432816,GSM5965598,Staphylococcus,USA300,Staphylococcusaureus-USA300,GCF_022226995.1


In [2]:
import pandas as pd
data = '../data_refs/'
meta = pd.read_csv(f'{data}SraRunTable_RNAseq_bacteria_oxfordnanopore.csv')
meta

run_to_org = dict(zip(meta.Run, meta.Organism)) #meta[meta['Run'] == 'SRR16009459']['Organism']
run_to_strain = dict(zip(meta.Run, meta.strain))

mapping_dir = '../output/mapping/'
runs = meta['Run'].values
mapping = []
columns = ['run', 'Organism', 'strain', 'reference_type', 'total', 'secondary', 'supplementary', 'duplicates', 'mapped', 'percent']
for run in runs:
    with open(f'{mapping_dir}{run}_minimap2_transcriptomic-quality.txt', 'r') as f:
        row = [run, run_to_org[run], run_to_strain[run], 'transcriptome']
        c = 0
        for line in f:
            line = line.split(' ')
            if c < 4:
                row.append(int(line[0]))
            elif c == 4:
                mapped = int(line[0])
                perc = float(line[4].split("(")[1].split('%')[0])
                row.append(mapped)
                row.append(perc)
                break
            c += 1
    mapping.append(row)
    with open(f'{mapping_dir}{run}_minimap2_genome-quality.txt', 'r') as f:
        row = [run, run_to_org[run], run_to_strain[run], 'genome']
        c = 0
        for line in f:
            line = line.split(' ')
            if c < 4:
                row.append(int(line[0]))
            elif c == 4:
                mapped = int(line[0])
                perc = float(line[4].split("(")[1].split('%')[0])
                row.append(mapped)
                row.append(perc)
                break
            c += 1
    mapping.append(row)
    
mapping_df = pd.DataFrame(data=mapping, columns=columns)
runs = meta[meta['Run'].isin(mapping_df[mapping_df['percent'] > 70]['run'].values)]
runs

,Run,Assay Type,AvgSpotLen,Bases,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,DATASTORE provider,...,tissue,geographic_location_(country_and/or_sea),host_health_state,host_scientific_name,Host_disease,BIOMATERIAL_PROVIDER,media,Note,treatment,type-material
0,SRR13212615,RNA-Seq,1096,1522106869,SAMN17023022,1454555653,"NEW ENGLAND BIOLABS, INC",public,"fastq,sra","s3,ncbi,gs",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR13212616,RNA-Seq,979,701284083,SAMN17023022,674896977,"NEW ENGLAND BIOLABS, INC",public,"sra,fastq","ncbi,s3,gs",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR13212618,RNA-Seq,717,137491381,SAMN17023022,135168447,"NEW ENGLAND BIOLABS, INC",public,"fastq,sra","gs,ncbi,s3",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR13212623,RNA-Seq,1096,1522106774,SAMN17023022,1454222338,"NEW ENGLAND BIOLABS, INC",public,"fastq,sra","ncbi,gs,s3",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR13212624,RNA-Seq,979,701284088,SAMN17023022,674731029,"NEW ENGLAND BIOLABS, INC",public,"fastq,sra","ncbi,s3,gs",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,SRR16608236,RNA-Seq,822,851534610,SAMN22630217,626320528,OSAKA UNIVERSTIY,public,"fastq,sra","ncbi,s3,gs",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,SRR16608237,RNA-Seq,879,1002194957,SAMN22630216,732648027,OSAKA UNIVERSTIY,public,"fastq,sra","ncbi,gs,s3",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,SRR16608238,RNA-Seq,873,1519514091,SAMN22630215,1107523038,OSAKA UNIVERSTIY,public,"fastq,sra","ncbi,s3,gs",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,SRR18432816,RNA-Seq,380,840405187,SAMN26871989,709489358,GEO,public,"fastq,sra","ncbi,gs,s3",...,NaN,NaN,NaN,NaN,NaN,NaN,TSB,NaN,Sample harvested at OD600 = 3 after growth in TSB,NaN


In [3]:
mapping_df['Organism'].value_counts()

Listeria monocytogenes                  58
Escherichia coli                        28
Streptococcus mutans                    24
Xylella fastidiosa subsp. fastidiosa    22
Pseudomonas aeruginosa UCBPP-PA14       12
Klebsiella pneumoniae                    8
Vibrio parahaemolyticus                  8
Pandoraea fibrosis                       4
Staphylococcus aureus                    4
Name: Organism, dtype: int64

In [4]:
run_to_ass['Name'].value_counts()

Listeriamonocytogenes-ref              29
Escherichiacoli-K_12                   12
Streptococcusmutans-B04Sm5             12
Xylellafastidiosa-stags_leap           11
Pseudomonasaeruginosa-UCBPP_PA14        6
Vibrioparahaemolyticus-RIMD_2210633     4
Escherichiacoli-E2348_69                2
Staphylococcusaureus-USA300             2
Klebsiellapneumoniae-16_GR_13           1
Klebsiellapneumoniae-20_GR_12           1
Klebsiellapneumoniae-1_GR_13            1
Klebsiellapneumoniae-2_GR_12            1
Pandoraeafibrosis-6399                  1
Pandoraeafibrosis-7641                  1
Name: Name, dtype: int64

In [5]:
run_to_ass = run_to_ass[run_to_ass['Run'].isin(runs['Run'].values)]
# Group by the assembly id
assemblies = run_to_ass[run_to_ass['Name'] == 'Escherichiacoli-K_12']
assemblies = list(set(assemblies['AssemblyID'].values))
assemblies

['GCF_000005845.2']

In [6]:
# For each of the runs that maps to these do a pairwise differr comp

In [22]:
! differr --help

Usage: differr [OPTIONS]

  A script for detecting differential error rates in aligned Nanopore data

Options:
  -a, --cond-A-bams TEXT          BAM files for low modification sample. To
                                  include multiple BAMs you need to use the -a
                                  flag multiple times.  [required]
  -b, --cond-B-bams TEXT          BAM files for wild type/complemented line
                                  with normal modification levels  [required]
  -r, --reference-fasta TEXT      [required]
  -o, --output-bed TEXT           [required]
  -c, --raw-counts-hdf TEXT
  -f, --fdr-threshold FLOAT
  -p, --processes INTEGER
  -m, --max-depth INTEGER         Maximum depth for pysam pileup, default is a
                                  very large number (basically samples all
                                  reads)
  --normalise / --no-normalise    Whether to normalise counts by sample depth
                                  (counts per million) before perfor

In [23]:
run_to_ass

,Run,SampleName,Species,Strain,Name,AssemblyID
0,SRR13212615,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
1,SRR13212616,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
2,SRR13212618,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
3,SRR13212623,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
4,SRR13212624,01_Listeria_RNA_seq,Listeria,NaN,Listeriamonocytogenes-ref,GCF_000196035.1
...,...,...,...,...,...,...
79,SRR16608236,Bile1,Vibrio,RIMD,Vibrioparahaemolyticus-RIMD_2210633,GCF_000196095.1
80,SRR16608237,LB2,Vibrio,RIMD,Vibrioparahaemolyticus-RIMD_2210633,GCF_000196095.1
81,SRR16608238,LB1,Vibrio,RIMD,Vibrioparahaemolyticus-RIMD_2210633,GCF_000196095.1
82,SRR18432816,GSM5965598,Staphylococcus,USA300,Staphylococcusaureus-USA300,GCF_022226995.1


In [7]:
import os
genome_dir = '../genomes/'
run_to_gff = {}
run_to_fna = {}
ass_to_gff = {}
ass_to_ref = {}
run_to_assembly = dict(zip(run_to_ass['Run'], run_to_ass['AssemblyID'] ))
files = os.listdir(genome_dir)
for run in run_to_assembly:
    assembly = run_to_assembly[run]
    for f in files:
        if assembly in f and 'fna' in f and 'fai' not in f:
            run_to_fna[run] = f'{genome_dir}{f}'
            ass_to_ref[assembly] = f.replace('.fna', '_transcripts.fasta')
        elif assembly in f and 'gff' in f:
            run_to_gff[run] = f'{genome_dir}{f}'
            ass_to_gff[assembly] = f

In [8]:
import itertools
for ass in assemblies:
    runs = list(run_to_ass[run_to_ass['AssemblyID'] == ass]['Run'].values)
    # Make pairwise comparisons
    pairwise_comps = []
    for p in itertools.combinations(runs, 2):
        pairwise_comps.append(p)
    
    for p in pairwise_comps:
         print(p)
#         os.system(f'differr -a ../runs/{p[0]}.sorted.bam -b ../runs/{p[1]}.sorted.bam -r ../genomes/{ass_to_ref[ass]} -o ../differrs/{p[0]}-{p[1]}_differr.txt')

('SRR12090426', 'SRR12090427')
('SRR12090426', 'SRR12090428')
('SRR12090426', 'SRR12090429')
('SRR12090426', 'SRR12090430')
('SRR12090426', 'SRR12090431')
('SRR12090426', 'SRR12090432')
('SRR12090426', 'SRR12090433')
('SRR12090426', 'SRR12090434')
('SRR12090426', 'SRR12090435')
('SRR12090426', 'SRR18061002')
('SRR12090426', 'SRR18061005')
('SRR12090427', 'SRR12090428')
('SRR12090427', 'SRR12090429')
('SRR12090427', 'SRR12090430')
('SRR12090427', 'SRR12090431')
('SRR12090427', 'SRR12090432')
('SRR12090427', 'SRR12090433')
('SRR12090427', 'SRR12090434')
('SRR12090427', 'SRR12090435')
('SRR12090427', 'SRR18061002')
('SRR12090427', 'SRR18061005')
('SRR12090428', 'SRR12090429')
('SRR12090428', 'SRR12090430')
('SRR12090428', 'SRR12090431')
('SRR12090428', 'SRR12090432')
('SRR12090428', 'SRR12090433')
('SRR12090428', 'SRR12090434')
('SRR12090428', 'SRR12090435')
('SRR12090428', 'SRR18061002')
('SRR12090428', 'SRR18061005')
('SRR12090429', 'SRR12090430')
('SRR12090429', 'SRR12090431')
('SRR120

In [9]:
import os

dones = os.listdir('../differrs/')
todo_pairwise_comps = []
for p in pairwise_comps:
    if f'{p[0]}-{p[1]}_differr.txt' not in dones:
        todo_pairwise_comps.append(p)
        
len(todo_pairwise_comps)

1

In [12]:
todo_pairwise_comps

[('SRR18061002', 'SRR18061005')]

In [11]:
for p in todo_pairwise_comps:
    os.system(f'differr -a ../runs/{p[0]}.sorted.bam -b ../runs/{p[1]}.sorted.bam -r ../genomes/{ass_to_ref[ass]} -o ../differrs/{p[0]}-{p[1]}_differr.txt')


In [ ]:
from multiprocessing.dummy import Pool as ThreadPool
import time

pool = ThreadPool(10)
def run_differr(p):
    start_time = time.time()
    os.system(f'differr -a ../runs/{p[0]}.sorted.bam -b ../runs/{p[1]}.sorted.bam -r ../genomes/{ass_to_ref[ass]} -o ../differrs/{p[0]}-{p[1]}_differr.txt')
    print(p[0], p[1], time.time() - start_time)
results = pool.map(run_differr, pairwise_comps)

SRR12090426 SRR12090431 8317.402362823486
SRR12090426 SRR12090435 8330.892300367355
SRR12090426 SRR12090429 8466.323043107986
SRR12090426 SRR12090433 8665.718384027481
SRR12090427 SRR12090429 9823.660451889038
SRR12090427 SRR12090435 10337.222225427628
SRR12090427 SRR12090433 10392.66998910904
SRR12090426 SRR12090427 10422.965144872665
SRR12090427 SRR12090431 10724.460843324661
SRR12090426 SRR18061002 6353.305411338806
SRR12090426 SRR12090432 8762.288733243942
SRR12090426 SRR12090434 8495.181030988693
SRR12090426 SRR12090430 9301.804865598679
SRR12090427 SRR18061002 7476.335870265961
SRR12090426 SRR18061005 18197.32166481018
SRR12090427 SRR12090434 9945.002958059311
SRR12090427 SRR12090430 10779.975956916809
SRR12090426 SRR12090428 10525.690552949905
SRR12090427 SRR12090432 10253.605538845062
SRR12090428 SRR18061002 7332.118495941162
SRR12090429 SRR18061002 5059.6106288433075
SRR12090428 SRR12090432 10127.752444505692
SRR12090428 SRR12090434 9663.305072784424
SRR12090428 SRR12090430 10

SRR12090426 SRR12090431 8317.402362823486
SRR12090426 SRR12090435 8330.892300367355
SRR12090426 SRR12090429 8466.323043107986
SRR12090426 SRR12090433 8665.718384027481
SRR12090427 SRR12090429 9823.660451889038
SRR12090427 SRR12090435 10337.222225427628
SRR12090427 SRR12090433 10392.66998910904
SRR12090426 SRR12090427 10422.965144872665
SRR12090427 SRR12090431 10724.460843324661
SRR12090426 SRR18061002 6353.305411338806
SRR12090426 SRR12090432 8762.288733243942
SRR12090426 SRR12090434 8495.181030988693
SRR12090426 SRR12090430 9301.804865598679
SRR12090427 SRR18061002 7476.335870265961
SRR12090426 SRR18061005 18197.32166481018
SRR12090427 SRR12090434 9945.002958059311
SRR12090427 SRR12090430 10779.975956916809
SRR12090426 SRR12090428 10525.690552949905
SRR12090427 SRR12090432 10253.605538845062
SRR12090428 SRR18061002 7332.118495941162
SRR12090429 SRR18061002 5059.6106288433075
SRR12090428 SRR12090432 10127.752444505692
SRR12090428 SRR12090434 9663.305072784424
SRR12090428 SRR12090430 10583.862995624542
SRR12090429 SRR12090432 7555.371153831482
SRR12090429 SRR12090434 7261.854347229004
SRR12090429 SRR12090430 8067.599304676056
SRR12090427 SRR12090428 12994.371564626694
SRR12090427 SRR18061005 19595.63259100914
SRR12090429 SRR12090435 7189.741761922836
SRR12090429 SRR12090431 7521.28968834877
SRR12090429 SRR12090433 7817.803208112717
SRR12090428 SRR12090435 9987.705408334732
SRR12090428 SRR12090433 10658.590497732162
SRR12090428 SRR12090431 10431.118582963943
SRR12090430 SRR12090431 8581.038507938385
SRR12090429 SRR18061005 17485.519361019135
SRR12090428 SRR12090429 9456.012546300888
SRR12090430 SRR12090435 8169.540106058121
SRR12090430 SRR12090433 9032.298450946808
SRR12090428 SRR18061005 19795.10078048706
SRR12090431 SRR12090435 9305.673272132874
SRR12090431 SRR12090433 9841.757977962494
SRR12090430 SRR12090432 8407.490545988083
SRR12090430 SRR18061002 5870.963866949081
SRR12090432 SRR18061002 6484.207571268082
SRR12090432 SRR12090434 9228.84089255333
SRR12090431 SRR18061002 5546.234978914261
SRR12090430 SRR12090434 8410.37084531784
SRR12090433 SRR12090434 9081.305979967117
SRR12090433 SRR18061002 6006.029147863388
SRR12090430 SRR18061005 18594.689717769623
SRR12090431 SRR12090434 8243.707969427109
SRR12090431 SRR18061005 17987.567662477493